# First Team

## Students:
Tugba Ceren Turhan,201ADB004\
Sofiia Azhniuk , 221ADB207



## Description
### Game description 

#### Before the start
- generate 5 numbers in the range of 10000 to 20000
- number chosen by the human player is given.
- Both players have 0 points. 
- The game uses a game bank, which is initially equal to 0.


#### How to play
Each player divide the current number by 2 or 3. 
can be divided only if the result is a whole number
if an even number, then the player gets 1 point\
if the number is odd, then his points are reduced by 1 point. \
if a number ending with 0 or 5 is obtained, then 1 point is added to the bank


#### The game ends when the number 2 or 3 is acquired. 
The player after whose turn the number 2 is acquired empties the bank by adding bank points to his points. \
The player with the most points at the end of the game wins. 

The result is a draw if the number of points is equal. 

### Additional software requirements 

Numbers are initially divisible by both 3 and 2\
The human player chooses which generated numbers he wants to start the game with. 


In [1]:
# import needed libraries
import random
import time
import pygame

#function to generate 5 random numbers
def get_5_random_numbers():
    def gen_number():
        number = 1
        while number < 10000:
            number *= random.randint(2, 3)
            if number > 20000:
                number = 1

        return number

    numbers = []
    while len(numbers) < 5:
        number = gen_number()
        if number not in numbers:
            numbers.append(number)

    return numbers



pygame 2.5.2 (SDL 2.28.3, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Classes

In [2]:

class Node:
    def __init__(self, number, ai_hum_scores, bank, isHumanTurn, divisor):
        self.number = number
        self.ai_hum_scores = ai_hum_scores.copy()
        self.bank = bank
        self.isHumanTurn = isHumanTurn
        self.children = []
        self.divisor = divisor



class Game_tree:
    def __init__(self, number, isHumanTurn):
        self.current_node = Node(number, [0, 0], 0, not isHumanTurn, 1)
        self.create_tree(self.current_node)

    def check_score(self, node):
        if node.number in [2, 3]:
            node.ai_hum_scores[node.isHumanTurn] += node.bank
            node.bank = 0
            return node

        if node.number % 2 == 0:
            node.ai_hum_scores[node.isHumanTurn] -= 1
        else:
            node.ai_hum_scores[node.isHumanTurn] += 1
        if node.number % 5 == 0:
            node.bank += 1
        return node

    def check_win(self, node):
        return node.number in [2, 3]

    def create_tree(self, node):
        if self.check_win(node):
            return

        for divisor in [2, 3]:
            if node.number % divisor == 0:
                new_number = node.number // divisor
                new_node = self.check_score(
                    Node(new_number, node.ai_hum_scores, node.bank, not node.isHumanTurn, divisor=divisor))
                node.children.append(new_node)
                self.create_tree(new_node)

    def make_move(self, divisor):
        if divisor == 2 or divisor == 3:
            for child in self.current_node.children:
                if child.divisor == divisor:
                    self.current_node = child
                    return True
        return False


class Game_logic:
    def __init__(self, number, isHumanTurn):
        self.gt = Game_tree(number, isHumanTurn)
        self.nodes_visited = 0

    def alpha_beta(self, node, depth, alpha, beta, maximizing_playerTurn):
        self.nodes_visited += 1

        if depth == 0 or self.gt.check_win(node):
            return node.ai_hum_scores[False] - node.ai_hum_scores[True]

        if maximizing_playerTurn:
            max_eval = float('-inf')
            for child in node.children:
                eval = self.alpha_beta(child, depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for child in node.children:
                eval = self.alpha_beta(child, depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def find_best_move_alpha_beta(self, node):
        best_move = None
        best_eval = float('-inf')
        alpha = float('-inf')
        beta = float('inf')
        for child in node.children:
            eval = self.alpha_beta(child, 4, alpha, beta, False)
            if eval > best_eval:
                best_eval = eval
                best_move = child
        return best_move

    def minimax(self, node, maximizing_playerTurn):
        self.nodes_visited += 1
        if node.children == []:
            return node.ai_hum_scores[False] - node.ai_hum_scores[True]

        if maximizing_playerTurn:
            best_value = float('-inf')
            for child in node.children:
                value = self.minimax(child, False)
                best_value = max(best_value, value)
            return best_value
        else:
            best_value = float('inf')
            for child in node.children:
                value = self.minimax(child, True)
                best_value = min(best_value, value)
            return best_value

    def find_best_move_minimax(self, node):
        best_value = float('-inf')
        best_move = None
        for child in node.children:
            value = self.minimax(child, False)
            if value > best_value:
                best_value = value
                best_move = child
        return best_move


class Player:
    def __init__(self):
        self.score = 0

    def add_score(self, score):
        self.score += score


### GUI Part

In [6]:
# Game parameters
chosen_number = random.choice(get_5_random_numbers())  # Randomly select a number

# Game objects
player_score = 0
ai_score = 0
bank = 0


BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Initialize Pygame
pygame.init()

# Set the dimensions of the screen
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Divide and Conquer")

# Dialog parameters
DIALOG_WIDTH = 400
DIALOG_HEIGHT = 300
DIALOG_X = (SCREEN_WIDTH - DIALOG_WIDTH) // 2
DIALOG_Y = (SCREEN_HEIGHT - DIALOG_HEIGHT) // 2
DIALOG_PADDING = 20

SAVE_BUTTON_WIDTH = 75
SAVE_BUTTON_HEIGHT = 30
SAVE_BUTTON_X = DIALOG_X + DIALOG_WIDTH // 2 - SAVE_BUTTON_WIDTH // 2
SAVE_BUTTON_Y = DIALOG_Y + DIALOG_HEIGHT - 80
save_button_rect = pygame.Rect(SAVE_BUTTON_X, SAVE_BUTTON_Y, SAVE_BUTTON_WIDTH, SAVE_BUTTON_HEIGHT)

# Fonts
font = pygame.font.Font(None, 36)
checkbox_font = pygame.font.Font(None, 24)


whosfirst = True
algo = "Alpha-Beta"
game_started = False

game_tree = Game_tree(chosen_number, whosfirst)
game_logic = Game_logic(chosen_number, whosfirst)
player1 = Player()
player2 = Player()
current_player = player1 if whosfirst else player2
opponent_player = player2 if whosfirst else player1


def draw_text(text, font, color, surface, x, y):
    textobj = font.render(text, 1, color)
    textrect = textobj.get_rect()
    textrect.topleft = (x, y)
    surface.blit(textobj, textrect)


def draw_checkbox(checkbox_text, font, color, surface, x, y, checkbox_width=20, checkbox_height=20, checkbox_padding=5,
                  checked=False):
    text_surface = font.render(checkbox_text, True, color)
    text_rect = text_surface.get_rect(topleft=(x + checkbox_width + checkbox_padding, y))
    pygame.draw.rect(surface, BLACK, (x, y, checkbox_width, checkbox_height), 2)
    surface.blit(text_surface, text_rect)
    if checked:
        pygame.draw.line(surface, BLACK, (x + 2, y + 2), (x + checkbox_width - 2, y + checkbox_height - 2), 2)
        pygame.draw.line(surface, BLACK, (x + checkbox_width - 2, y + 2), (x + 2, y + checkbox_height - 2), 2)
    return pygame.Rect(x, y, checkbox_width + text_rect.width + checkbox_padding, checkbox_height)


# Function to draw the dialog window
def draw_dialog():
    pygame.draw.rect(SCREEN, WHITE, (DIALOG_X, DIALOG_Y, DIALOG_WIDTH, DIALOG_HEIGHT))
    draw_text("Game Settings", font, BLACK, SCREEN, DIALOG_X + DIALOG_PADDING, DIALOG_Y + DIALOG_PADDING)
    human_turn_checkbox = draw_checkbox("Human's Turn", checkbox_font, BLACK, SCREEN, DIALOG_X + DIALOG_PADDING,
                                        DIALOG_Y + DIALOG_PADDING * 3, checked=whosfirst)
    ai_turn_checkbox = draw_checkbox("AI's Turn", checkbox_font, BLACK, SCREEN, DIALOG_X + DIALOG_PADDING,
                                     DIALOG_Y + DIALOG_PADDING * 4, checked=not whosfirst)
    algo_checkbox = draw_checkbox(f"Algorithm: {algo}", checkbox_font, BLACK, SCREEN, DIALOG_X + DIALOG_PADDING,
                                  DIALOG_Y + DIALOG_PADDING * 5)
    pygame.draw.rect(SCREEN, BLACK, save_button_rect)
    draw_text("Save", font, WHITE, SCREEN, SAVE_BUTTON_X + 10, SAVE_BUTTON_Y + 5)
    return human_turn_checkbox, ai_turn_checkbox, algo_checkbox


def start_game():
    global game_tree, game_logic, current_player
    game_tree = Game_tree(chosen_number, whosfirst)
    game_logic = Game_logic(chosen_number, whosfirst)
    current_player = player1 if whosfirst else player2

def display_winner(winner, nodes_explored):
    pygame.display.set_caption("Game Over")
    dialog_font = pygame.font.Font(None, 48)
    if winner == "AI":
        message = "AI wins!"
    elif winner == "Human":
        message = "Human player wins!"
    else:
        message = "It's a draw!"
    text = dialog_font.render(message, True, BLACK)
    text_rect = text.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))

    nodes_text = f"Nodes Explored by AI: {nodes_explored}"
    nodes_text_rendered = dialog_font.render(nodes_text, True, BLACK)
    nodes_text_rect = nodes_text_rendered.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 50))

    SCREEN.blit(text, text_rect)
    SCREEN.blit(nodes_text_rendered, nodes_text_rect)

    pygame.display.flip()
    pygame.time.delay(3000)
    pygame.quit()


# Main game loop
running = True
human_turn_checkbox, ai_turn_checkbox, algo_checkbox = draw_dialog()
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                mouse_pos = event.pos
                if human_turn_checkbox.collidepoint(mouse_pos):
                    whosfirst = True
                elif ai_turn_checkbox.collidepoint(mouse_pos):
                    whosfirst = False
                elif algo_checkbox.collidepoint(mouse_pos):
                    algo = "Alpha-Beta" if algo == "Minimax" else "Minimax"
                elif save_button_rect.collidepoint(mouse_pos):
                    start_game()
                    game_started = True
                    current_player = player1 if whosfirst else player2
                human_turn_checkbox, ai_turn_checkbox, algo_checkbox = draw_dialog()

    if game_started:
        SCREEN.fill(WHITE)
        if current_player == player1:
            # Human player's turn
            mouse_pos = pygame.mouse.get_pos()
            if pygame.mouse.get_pressed()[0]:
                if 200 < mouse_pos[0] < 400 and 200 < mouse_pos[1] < 300:
                    divisor = 2
                elif 400 < mouse_pos[0] < 600 and 200 < mouse_pos[1] < 300:
                    divisor = 3
                else:
                    continue

                if game_tree.make_move(divisor):
                    player_score += -1 if game_tree.current_node.number % 2 != 0 else 1
                    if game_tree.current_node.number % 5 == 0:
                        game_tree.current_node.bank += 1
                    current_player = player2  # Switch to AI player after human's turn
        else:
            # AI's turn
            start_time = time.time()
            if algo == "Alpha-Beta":
                best_move = game_logic.find_best_move_alpha_beta(game_tree.current_node)
            else:
                best_move = game_logic.find_best_move_minimax(game_tree.current_node)
            end_time = time.time()
            evaluation_time = end_time - start_time

            if game_tree.current_node.number % 2 == 0 and game_tree.current_node.number % 3 == 0:
                divisor = best_move.divisor
            else:
                divisor = 2 if game_tree.current_node.number % 2 == 0 else 3
            game_tree.make_move(divisor)
            ai_score += -1 if game_tree.current_node.number % 2 != 0 else 1
            if game_tree.current_node.number % 5 == 0:
                game_tree.current_node.bank += 1
            current_player = player1  # Switch to human player after AI's turn
            time.sleep(1)

            # Check if the game is over
            if game_tree.current_node.number in [2, 3]:
                if player_score > ai_score:
                    display_winner("Human", game_logic.nodes_visited)
                elif ai_score > player_score:
                    display_winner("AI", game_logic.nodes_visited)
                else:
                    display_winner("Draw", game_logic.nodes_visited)

        # Draw buttons
        pygame.draw.rect(SCREEN, GREEN, (200, 200, 200, 100))
        pygame.draw.rect(SCREEN, RED, (400, 200, 200, 100))

        draw_text("Divide by 2", font, BLACK, SCREEN, 250, 235)
        draw_text("Divide by 3", font, BLACK, SCREEN, 450, 235)

        # Draw game state
        draw_text(f"Number: {game_tree.current_node.number}", font, BLACK, SCREEN, 50, 50)
        draw_text(f"AI Score: {ai_score}", font, BLACK, SCREEN, 50, 100)
        draw_text(f"Player Score: {player_score}", font, BLACK, SCREEN, 50, 150)
        draw_text(f"Bank: {game_tree.current_node.bank}", font, BLACK, SCREEN, 50, 200)
        draw_text(f"Algorithm: {algo}", font, BLACK, SCREEN, 600, 50)
        draw_text(f"Player's Turn: {'Human' if current_player == player1 else 'AI'}", font, BLACK, SCREEN, 600, 100)
        # draw_text(f"Evaluation Time: {evaluation_time:.4f} seconds", font, BLACK, SCREEN, 600, 150)

    pygame.display.flip()

pygame.quit()


In [ ]:
import random
import time


class Node:
    def __init__(self, number, ai_hum_scores, bank, isHumanTurn, divisor):
        self.number = number
        self.ai_hum_scores = ai_hum_scores.copy()
        self.bank = bank
        self.isHumanTurn = isHumanTurn
        self.children = []
        self.divisor = divisor


class Game_tree:
    def __init__(self, number, isHumanTurn):
        self.current_node = Node(number, [0, 0], 0, not isHumanTurn, 1)
        self.create_tree(self.current_node)

    def check_score(self, node):
        if node.number in [2, 3]:
            node.ai_hum_scores[node.isHumanTurn] += node.bank
            node.bank = 0
            return node

        if node.number % 2 == 0:
            node.ai_hum_scores[node.isHumanTurn] -= 1
        else:
            node.ai_hum_scores[node.isHumanTurn] += 1
        if node.number % 5 == 0:
            node.bank += 1
        return node

    def check_win(self, node):
        return node.number in [2, 3]

    def create_tree(self, node):
        if self.check_win(node):
            return

        for divisor in [2, 3]:
            if node.number % divisor == 0:
                new_number = node.number // divisor
                new_node = self.check_score(
                    Node(new_number, node.ai_hum_scores, node.bank, not node.isHumanTurn, divisor=divisor))
                node.children.append(new_node)
                self.create_tree(new_node)

    def make_move(self, divisor):
        if divisor == 2 or divisor == 3:
            for child in self.current_node.children:
                if child.divisor == divisor:
                    self.current_node = child
                    return True
        return False

    def print_tree(self, node):
        print("num: ", node.number, "\tBank score", node.bank, "\tai and human score: ", node.ai_hum_scores,
              "\thuman turn: ", node.isHumanTurn, '\tLevel: ', node.level, '\tDivisor:', node.divisor)
        for i in node.children:
            self.print_tree(i)


class Game_logic:
    def __init__(self, number, isHumanTurn):
        self.gt = Game_tree(number, isHumanTurn)
        self.nodes_visited = 0

    def alpha_beta(self, node, depth, alpha, beta, maximizing_playerTurn):
        self.nodes_visited += 1

        if depth == 0 or self.gt.check_win(node):
            return node.ai_hum_scores[False] - node.ai_hum_scores[True]

        if maximizing_playerTurn:
            max_eval = float('-inf')  # Initialize the max evaluation to negative infinity
            for child in node.children:  # Iterate through the children nodes
                eval = self.alpha_beta(child, depth - 1, alpha, beta,
                                       False)  # Recursively call alpha_beta for the child node
                max_eval = max(max_eval,
                               eval)  # Update the max_eval to the maximum of its current value and the evaluation of the child node
                alpha = max(alpha,
                            eval)  # Update alpha to the maximum of its current value and the evaluation of the child node
                if beta <= alpha:  # If beta is less than or equal to alpha, prune the search
                    break
            return max_eval
        else:
            min_eval = float('inf')  # Initialize the min evaluation to positive infinity
            for child in node.children:  # Iterate through the children nodes
                eval = self.alpha_beta(child, depth - 1, alpha, beta,
                                       True)  # Recursively call alpha_beta for the child node
                min_eval = min(min_eval,
                               eval)  # Update the min_eval to the minimum of its current value and the evaluation of the child node
                beta = min(beta,
                           eval)  # Update beta to the minimum of its current value and the evaluation of the child node
                if beta <= alpha:  # If beta is less than or equal to alpha, prune the search
                    break
            return min_eval

    def find_best_move_alpha_beta(self, node):
        best_move = None
        best_eval = float('-inf')
        alpha = float('-inf')
        beta = float('inf')
        for child in node.children:
            eval = self.alpha_beta(child, 4, alpha, beta, False)
            if eval > best_eval:
                best_eval = eval
                best_move = child
        return best_move

    def minimax(self, node, maximizing_playerTurn):
        self.nodes_visited += 1
        if node.children == []:
            return node.ai_hum_scores[False] - node.ai_hum_scores[True]

        if maximizing_playerTurn:
            best_value = float('-inf')
            for child in node.children:
                value = self.minimax(child, False)
                best_value = max(best_value, value)
            return best_value
        else:
            best_value = float('inf')
            for child in node.children:
                value = self.minimax(child, True)
                best_value = min(best_value, value)
            return best_value

    def find_best_move_minimax(self, node):
        best_value = float('-inf')
        best_move = None
        for child in node.children:
            value = self.minimax(child, False)
            if value > best_value:
                best_value = value
                best_move = child
        return best_move


class Player:
    def __init__(self):
        self.score = 0

    def add_score(self, score):
        self.score += score


def play_game(chosen_number, is_human_first):
    gt = Game_tree(chosen_number, is_human_first)
    gl = Game_logic(chosen_number, is_human_first)
    player1 = Player()
    player2 = Player()

    current_player = player1 if is_human_first else player2
    opponent_player = player2 if is_human_first else player1

    if is_human_first:
        print("Human player goes first.")
    else:
        print("AI goes first.")
    algo = int(input("Choose {0|1} whether u want to use Alpha-Beta | Minimax: "))
    print("U have Chosen Minimax") if algo == 1 else print("U have Chosen Alpha-Beta")
    while not gt.current_node.number in [2, 3]:
        if current_player == player1:
            divisor = int(input("Enter divisor 2|3: "))
            if gt.current_node.number % divisor == 0:
                gt.make_move(divisor)
                player1.add_score(-1 if gt.current_node.number % 2 != 0 else 1)
                if gt.current_node.number % 5 == 0:
                    gt.current_node.bank += 1
            else:
                print("Impossible to divide on following divisor!")

        else:
            start_time = time.time()
            if algo == 0:
                best_move = gl.find_best_move_alpha_beta(gt.current_node)
            else:
                best_move = gl.find_best_move_minimax(gt.current_node)
            end_time = time.time()
            evaluation_time = end_time - start_time  # Calculate evaluation time

            if gt.current_node.number % 2 == 0 and gt.current_node.number % 3 == 0:
                divisor = best_move.divisor
            else:
                divisor = 2 if gt.current_node.number % 2 == 0 else 3
            gt.make_move(divisor)
            player2.add_score(-1 if gt.current_node.number % 2 != 0 else 1)
            if gt.current_node.number % 5 == 0:
                gt.current_node.bank += 1
            print(f"AI has chosen divisor {divisor}, Evaluation Time: {evaluation_time:.4f} seconds")

        print(
            f"Number: {gt.current_node.number}, AI Score: {player2.score}, Player Score: {player1.score}, Bank: {gt.current_node.bank}")

        current_player, opponent_player = opponent_player, current_player

        if not gt.current_node.number in [2, 3] and current_player == player1:
            continue_game = input("Press Enter to continue...")

    if gt.current_node.number == 2 or gt.current_node.number == 3:
        if current_player == player2:
            player2.add_score(gt.current_node.bank)
            if player2.score > player1.score:
                print("AI wins!")
            elif player2.score < player1.score:
                print("Human player wins!")
            else:
                print("It's a draw!")
        else:
            player1.add_score(gt.current_node.bank)
            if player1.score > player2.score:
                print("Human player wins!")
            elif player1.score < player2.score:
                print("AI wins!")
            else:
                print("It's a draw!")
    else:
        if current_player == player2:
            print("AI wins!")
        else:
            print("Human player wins!")
    print("Nodes visited: ", gl.nodes_visited)


def get_5_random_numbers():
    def gen_number():
        number = 1
        while number < 10000:
            number *= random.randint(2, 3)
            if number > 20000:
                number = 1

        return number

    numbers = []
    while len(numbers) < 5:
        number = gen_number()
        if number not in numbers:
            numbers.append(number)

    return numbers


chosen_number = random.choice(get_5_random_numbers())
# chosen_number = 14334
# chosen_number = 11664
whosfirst = int(input("Enter {0,1} for Human, AI to start: "))
des = True if whosfirst == 0 else False
print("Chosen number: ", chosen_number)
play_game(chosen_number, des)

# gt = Game_tree(chosen_number, True)
# gt.print_tree(gt.current_node)